# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# import os

# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv())
from google.colab import userdata

OPENAI_API_KEY  = userdata.get('Ironhack-GPT')
HUGGINGFACEHUB_API_TOKEN = userdata.get('HF')

In [ ]:
#!pip install pandas

In [6]:
import pandas as pd
df = pd.read_csv('/content/Data.csv')

In [ ]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [10]:
!pip install langchain_openai
!pip install langchain

from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 6.5 MB/s eta 0:00:00


In [32]:
#Replace None by your own value and justify
llm = ChatOpenAI(model_name="gpt-4-turbo", api_key = OPENAI_API_KEY, temperature=0.9)

In [27]:
prompt = ChatPromptTemplate.from_template( 'Write me a description on {product}'
)

In [28]:
chain = LLMChain(llm=llm, prompt=prompt)

In [29]:
product = 'Luxury Air Mattress'
response = chain.run(product)
response

"A luxury air mattress elevates the traditional concept of an inflatable bed by combining state-of-the-art technology, superior materials, and refined design to deliver an unparalleled sleep experience. Unlike standard air mattresses, a luxury model focuses intently on comfort, durability, and convenience, making it a top choice for discerning users seeking a temporary or semi-permanent sleeping solution without compromising on quality.\n\nThe core of a luxury air mattress lies in its sophisticated internal structure, often featuring multiple air chambers rather than a single inflatable section. These chambers are independently adjustable, allowing users to customize the firmness of the mattress according to their personal preference or even adjust different sides to different firmness levels, perfect for couples with varying comfort needs.\n\nCrafted from high-grade, puncture-resistant PVC or textile-reinforced urethane, these mattresses are not only tough but also designed to mimic t

## SimpleSequentialChain

In [30]:
from langchain.chains import SimpleSequentialChain

In [35]:
#llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template( 'List 3 brands that sell {product}'
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [36]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template( 'out of the {output}, list 5 other products these brands sell'
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [37]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [38]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
As of my last update, there are specific brands known for their high-end or luxury air mattresses that often include features like adjustable firmness, built-in pumps, and superior materials. Three brands that sell luxury air mattresses include:

1. **Serta** - Known for both innerspring and foam mattresses, Serta also offers air mattresses that are considered high quality with features like neverFLAT pump technology, which helps maintain air pressure throughout the night.

2. **SoundAsleep** - While perhaps not traditionally "luxury" in the classic sense, SoundAsleep's Dream Series air mattress is often praised for its comfort and durability, making it a premium choice in the air mattress category.

3. **Intex** - Offers a range of air mattresses, including higher-end models like the Ultra Plush Airbed, which features a built-in pump and a design focused on providing a luxurious sleeping experience.

These brands are known for their reli

"Each of these brands, known for their air mattresses, also offers a variety of other sleep-related products that cater to diverse customer needs. Here are some additional products sold by Serta, SoundAsleep, and Intex:\n\n### Serta\n1. **Memory Foam Mattresses** - These mattresses include multiple layers of foam that contour to the body to reduce pressure points and provide support.\n2. **Hybrid Mattresses** - Combining innerspring coils with memory foam, these mattresses offer a balance of comfort and support.\n3. **Bed Frames** - Serta sells various bed frames that are designed to support their mattresses optimally.\n4. **Pillows** - They offer a range of pillows in different materials and firmness levels to complement their mattresses.\n5. **Adjustable Foundations** - These allow users to adjust the position of their mattress for optimal comfort and support, beneficial for reading, watching TV, or sleeping.\n\n### SoundAsleep\n1. **Camping Series Air Mattresses** - Designed specifi

**Repeat the above twice for different products**

## SequentialChain

In [45]:
from langchain.chains import SequentialChain

In [48]:
first_prompt = ChatPromptTemplate.from_template('translate {review} from english to lithuanian'
)

chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key='LithuanianTranslation' #Give a name to your output
                    )


In [49]:
second_prompt = ChatPromptTemplate.from_template('summarize {LithuanianTranslation}'
)

chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key='LithuanianTranslationSummary' #give a name to this output
                    )


In [50]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template('translate the {LithuanianTranslationSummary} back to english'
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="SummaryBackTranslation"
                      )


In [51]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template( 'Based on the original review and the translations, write a message > Original review: {review}, Back translation: {SummaryBackTranslation}'
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key='Final'
                     )


In [53]:
# overall_chain: input= Review
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=['review'],
    output_variables=['LithuanianTranslation', 'LithuanianTranslationSummary', 'SummaryBackTranslation', 'Final'],
    verbose=True
)

In [55]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'LithuanianTranslation': 'Aš manau, kad skonis prastas. Putos neišsilaiko, tai keista. Aš perkų tuos pačius prekyboje ir skonis yra daug geresnis... Senas siuntinys arba klastotė!?',
 'LithuanianTranslationSummary': 'Asmens komentaras apie produktą nurodo, kad skonis yra prastas ir putos neišsilaiko, kas yra neįprasta. Taip pat minima, kad tas pats produktas, pirktas prekyboje, paprastai skonis yra geresnis. Todėl kyla abejonių, ar gautas produktas yra senas ar galbūt netikras.',
 'SummaryBackTranslation': "The person's comment about the product indicates that the taste is poor and the foam does not last, which is unusual. It is also mentioned that the same product, when bought in stores, usually tastes better. Therefore, there are doubts whether the received product is old or possibly counterfeit.",
 'Final':

**Repeat the above twice for different products or reviews**

## Router Chain

In [56]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

In [57]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    }
]

In [58]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [60]:
#llm = ChatOpenAI(temperature=0)

In [61]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [62]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [63]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [64]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [65]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

In [66]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by an idealized physical body, known as a black body. A black body absorbs all incident electromagnetic radiation, regardless of frequency or angle of incidence. The radiation emitted by a black body is solely a function of its temperature.\n\nThe spectrum of black body radiation, which means the distribution of energy at different wavelengths, is determined by the body's temperature. The higher the temperature, the more total radiation emitted, and the peak of the radiation shifts to shorter wavelengths. This shift can be observed in everyday phenomena; for example, heating a metal rod in a fire will cause it to glow red and then white as it gets hotter.\n\nThe theoretical explanation of black body radiation was a significant challenge to classical physics, leading to the development of quantum mechanics. Max Planck, in 1900, proposed that energy is quantized and introduced the concept of the Planck constant to expl

In [67]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'Thank you for the kind words! To answer your question, \\(2 + 2\\) is simply the addition of the numbers 2 and 2. When these are added together, the result is \\(4\\).'

In [68]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


"Every cell in our body contains DNA because it is essential for carrying the genetic instructions used in the growth, development, functioning, and reproduction of all known organisms and many viruses. DNA sequences are like biological codes or blueprints that provide detailed instructions on how to build and maintain an organism. Here’s a breakdown of why DNA is crucial in every cell:\n\n1. **Genetic Blueprint:** DNA contains the information needed to build the various proteins and molecules necessary for a cell to function properly. These proteins include enzymes, structural components of the cell, and signaling molecules, all of which are essential for the physiological and biochemical functions of life.\n\n2. **Cell Function and Regulation:** The sequences within DNA determine not only the structure and function of the proteins but also when, where, and how these proteins are to be made. This regulation is crucial for making sure that cells behave appropriately, grow correctly, an

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [71]:
music_template = """You are music freak. \
You are great at answering questions about music and musicians in a musical\
way that oftern cites 80's hits. \

Here is a question:
{input}"""


movies_template = """You know your movies in and out \
You are great at making movie puns \
You are so good because you give some fun facts that are
unknown to most peaopl

Here is a question:
{input}"""

food_template = """You are a very good cook. \
you havea a great international taste and love to do fusion dishes.
you like to give out nice recipies and often send some youtube links for your favourite dishes

Here is a question:
{input}"""


In [76]:
prompt_infos = [
    {
        "name": "music",
        "description": "Good for answering questions about music",
        "prompt_template": music_template
    },
    {
        "name": "movies",
        "description": "Good for answering movies questions",
        "prompt_template": movies_template
    },
    {
        "name": "food",
        "description": "Good for answering food questions",
        "prompt_template": food_template
    }
]

In [77]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [78]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [79]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

In [80]:
chain.run("another one bites the dust?")



> Entering new MultiPromptChain chain...
music: {'input': "What is the song 'Another One Bites the Dust' about?"}
> Finished chain.


'"Another One Bites the Dust," by Queen, grooves to a beat that\'s hard to resist, and while its funky bass line might get you on your feet, the lyrics might make you ponder a bit. The song, released in 1980 on their album "The Game," packs a punch with its repetitive, catchy chorus and striking bass riff by John Deacon.\n\nContrary to its upbeat musical arrangement, the song’s theme dives into darker territories, exploring themes of defeat and survival. Although interpretations can vary, with some hearing echoes of confrontations and violent outcomes ("Out of the doorway the bullets rip / To the sound of the beat"), it can also be seen as a figurative expression of overcoming challenges or competitors—one by one succumbing or \'biting the dust\' as the protagonist moves ahead.\n\nThe song\'s vibe and narrative style somewhat leave the exact scenario to the listener’s imagination, but it’s clear why this track became a smash hit: it combines Queen\'s signature rock flair with irresisti

In [81]:
chain.run("tell me about falafel")



> Entering new MultiPromptChain chain...
food: {'input': 'tell me about falafel'}
> Finished chain.


"Thank, you for recognizing my passion for cooking and international flavors! Falafel is a delicious and popular Middle Eastern dish that's loved worldwide. It's traditionally made from ground chickpeas or fava beans, mixed with a variety of herbs and spices such as parsley, cilantro, cumin, and coriander. The mixture is then shaped into small balls or patties and deep-fried until golden brown, resulting in a crispy exterior with a soft, flavorful interior.\n\nFalafel is often served in a pita or flatbread with salads, pickled vegetables, hot sauce, and drizzled with tahini-based sauces. It can also be served over a bed of greens as a salad topping or as an appetizer with various dips like hummus and tzatziki.\n\nFor a fusion twist, you could try incorporating different herbs and spices into the falafel mix or serve them with a sauce inspired by another cuisine—like a curry mayo or a spicy Latin American salsa.\n\nIf you're interested in making falafel, there are plenty of great recipe

In [82]:
chain.run("I'm gonna become a godfather, what are the best impressions I can do?")



> Entering new MultiPromptChain chain...
movies: {'input': "What are some iconic impressions or quotes from 'The Godfather' movie that I can perform?"}
> Finished chain.


'"The Godfather" is a treasure trove of iconic lines and impressions that are instantly recognizable and perfect for performance. Here are some of the most famous quotes from the movie that you can use:\n\n1. **"I\'m going to make him an offer he can\'t refuse."** - Don Vito Corleone (Marlon Brando)\n   - This is perhaps the most famous line from the movie. It encapsulates the mix of menace and allure that the Godfather himself exudes. When delivering this line, remember to adopt a raspy voice like Brando’s and maintain a calm, authoritative demeanor.\n\n2. **"It\'s not personal, Sonny. It\'s strictly business."** - Michael Corleone (Al Pacino)\n   - A line that perfectly captures the cold, calculated approach of the Corleone family to the business of organized crime. When saying this, adopt a serious, focused tone.\n\n3. **"Leave the gun. Take the cannoli."** - Clemenza (Richard S. Castellano)\n   - This line mixes dark humor with the grim business of the Mafia. It\'s delivered after 

In [83]:
chain.run("danny devito")



> Entering new MultiPromptChain chain...
None: {'input': 'danny devito'}
> Finished chain.


'Danny DeVito is an American actor, producer, and director, known for his distinctive raspy voice, short stature, and comedic talent. Born on November 17, 1944, in Neptune Township, New Jersey, DeVito has had a distinguished career in film and television.\n\nOne of DeVito\'s most notable roles was as Louie De Palma in the television series "Taxi," which aired from 1978 to 1983. This role earned him a Golden Globe Award and an Emmy Award. He is also well-known for his role as Frank Reynolds on the FX and FXX sitcom "It\'s Always Sunny in Philadelphia," which he joined in its second season in 2006.\n\nIn addition to his work in television, DeVito has appeared in numerous films, including "One Flew Over the Cuckoo\'s Nest" (1975), "Throw Momma from the Train" (1987), "Twins" (1988), "Batman Returns" (1992), where he played the iconic villain The Penguin, and "Matilda" (1996), which he also directed.\n\nDeVito\'s directing credits also include "The War of the Roses" (1989) and "Hoffa" (199